In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
!pip install dagshub mlflow --quiet
import dagshub
import mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 46.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 73.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.0/684.0 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beha

In [3]:
dagshub.init(repo_owner='nkikn21', repo_name='House-Prices', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=2fed6266-7d11-4b0f-9519-3c1efc887937&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=9e27b042a22f098fa17d96eead363224ffa6003898e491dbef397c191312a4fe




Output()

Accessing as nkikn21

Initialized MLflow to track repo "nkikn21/House-Prices"

Repository nkikn21/House-Prices initialized!

In [18]:
df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_raw = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

In [19]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['SalePrice'])
y = df['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
test_ids = test_raw.pop('Id')
train_ids = X_train.pop('Id')
X_test_ids = X_test.pop('Id')

# Missings

In [21]:
cols_to_drop = ['Alley', 'PoolQC', 'Fence', 'MiscFeature']
for col in cols_to_drop:
    test_raw = test_raw.drop(col, axis=1)
    X_train = X_train.drop(col, axis=1)

masVn_median = X_train["MasVnrArea"].median() 
X_train["MasVnrArea"] = X_train["MasVnrArea"].fillna(masVn_median)
test_raw["MasVnrArea"] = test_raw["MasVnrArea"].fillna(masVn_median)

garage_median = X_train["GarageYrBlt"].median()
X_train["GarageYrBlt"] = X_train["GarageYrBlt"].fillna(garage_median)
test_raw["GarageYrBlt"] = test_raw["GarageYrBlt"].fillna(garage_median)
# X_train["GarageYrBlt"] = X_train["GarageYrBlt"].fillna(0)
# test_raw["GarageYrBlt"] = test_raw["GarageYrBlt"].fillna(0)

lot_mean = X_train["LotFrontage"].mean()
X_train["LotFrontage"] = X_train["LotFrontage"].fillna(lot_mean) 
test_raw["LotFrontage"] = test_raw["LotFrontage"].fillna(lot_mean) 
# lot_median = X_train['LotFrontage'].median()
# X_train["LotFrontage"] = X_train["LotFrontage"].fillna(lot_median)
# test_raw["LotFrontage"] = test_raw["LotFrontage"].fillna(lot_median)

object_missings = ["BsmtQual", "MasVnrType", "BsmtCond", "BsmtExposure", "BsmtFinType1",
                "BsmtFinType2", "Electrical", "FireplaceQu", "GarageType",
                "GarageFinish", "GarageQual", "GarageCond"]

for col in object_missings:
    cat_mean = X_train[col].mode()[0]
    X_train[col] = X_train[col].fillna(cat_mean)
    test_raw[col] = test_raw[col].fillna(cat_mean)

for col in test_raw.columns:
    if test_raw[col].isnull().any():
        if test_raw[col].dtype == 'object':
            test_raw[col] = test_raw[col].fillna(X_train[col].mode()[0])  
        else:
            test_raw[col] = test_raw[col].fillna(X_train[col].median()) 

cat_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']


**Model v3**

In [8]:
# for col in X_train.columns:
#     if X_train[col].isnull().any():
#         if X_train[col].dtype == 'object':
#             X_train[col] = X_train[col].fillna(X_train[col].mode()[0])  
#         else:
#             X_train[col] = X_train[col].fillna(X_train[col].median()) 
            
# for col in test_raw.columns:
#     if test_raw[col].isnull().any():
#         if test_raw[col].dtype == 'object':
#             test_raw[col] = test_raw[col].fillna(X_train[col].mode()[0])  
#         else:
#             test_raw[col] = test_raw[col].fillna(X_train[col].median()) 

In [22]:
missing_test = test_raw.isnull().sum()
missing_test = missing_test[missing_test > 0]
print(missing_test)


Series([], dtype: int64)


# Handle Categoricals

In [23]:
s = X_train[cat_cols].nunique()
threshold = 3
# threshold = 4
one_hot_columns = list(s[s <= threshold].index)
woe_columns = list(s[s > threshold].index)

***One_Hot***

In [24]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore', drop='first', sparse_output=False)

X_train_ohe = pd.DataFrame(
    ohe.fit_transform(X_train[one_hot_columns]),
    columns=ohe.get_feature_names_out(one_hot_columns),
    index=X_train.index
)

X_test_ohe = pd.DataFrame(
    ohe.transform(test_raw[one_hot_columns]),
    columns=ohe.get_feature_names_out(one_hot_columns),
    index=test_raw.index
)

X_train_final = X_train.drop(columns=one_hot_columns).join(X_train_ohe)
X_train = X_train_final
test_raw_final = test_raw.drop(columns=one_hot_columns).join(X_test_ohe)
test_raw = test_raw_final

***WOE***

In [25]:
from category_encoders.woe import WOEEncoder

threshold = y_train.median()
y_binary = (y_train > threshold).astype(int)

woe_encoder = WOEEncoder(cols=woe_columns)
woe_encoder.fit(X_train[woe_columns], y_binary)
X_train_woe = woe_encoder.transform(X_train[woe_columns])
test_raw_woe = woe_encoder.transform(test_raw[woe_columns])
test_raw_final = test_raw.drop(columns=woe_columns).join(test_raw_woe)
test_raw = test_raw_final
X_train_final = X_train.drop(columns=woe_columns).join(X_train_woe)
X_train = X_train_final


# Feature Selection

***Correlation Filter***

**Models v1 and v2**

In [57]:
cols_to_drop = ['TotRmsAbvGrd','Exterior2nd','1stFlrSF','SaleCondition','GarageArea','MasVnrType_BrkFace']
test_raw = test_raw.drop(columns=cols_to_drop)
X_train = X_train.drop(columns=cols_to_drop)

**Model v3**

In [14]:
cols_to_drop = ['ExterQual_Gd','MasVnrType_BrkFace','Exterior2nd']
test_raw = test_raw.drop(columns=cols_to_drop)
X_train = X_train.drop(columns=cols_to_drop)

# Final Prediction

**Linear Regression v1**

In [ ]:
# from sklearn.preprocessing import StandardScaler

# rfe_features = ['MSSubClass','OverallQual','OverallCond','MasVnrArea','BsmtFinSF1','GrLivArea','BsmtFullBath',
#                 'GarageCars','LandContour','Neighborhood','Condition2','BsmtQual','BsmtExposure','KitchenQual','SaleType']

# scaler = StandardScaler()

# X_train_scaled = pd.DataFrame(
#     scaler.fit_transform(X_train),
#     columns=X_train.columns
# )

# test_raw_scaled = pd.DataFrame(
#     scaler.transform(test_raw),  
#     columns=test_raw.columns
# )
# test_raw_final = test_raw_scaled[rfe_features]

# import mlflow
# logged_model = 'runs:/dda5795e8c97497ba67cdc4de68cee40/final_model'

# # Load model as a PyFuncModel.
# loaded_model = mlflow.pyfunc.load_model(logged_model)

# # Predict on a Pandas DataFrame.
# import pandas as pd
# loaded_model.predict(test_raw_final)

**Linear Regression v2**

In [58]:
# from sklearn.preprocessing import StandardScaler

# rfe_features = ['MSSubClass','OverallQual','OverallCond','MasVnrArea','BsmtUnfSF', 'TotalBsmtSF', 'GrLivArea',
#                 'GarageCars','LandContour','Neighborhood','Condition2','BsmtQual','BsmtExposure','KitchenQual','SaleType']

# scaler = StandardScaler()

# X_train_scaled = pd.DataFrame(
#     scaler.fit_transform(X_train),
#     columns=X_train.columns
# )

# test_raw_scaled = pd.DataFrame(
#     scaler.transform(test_raw),  
#     columns=test_raw.columns
# )
# test_raw_final = test_raw_scaled[rfe_features]

# import mlflow
# logged_model = 'runs:/c398f06a89614c47a509924d58678c78/final_model'

# # Load model as a PyFuncModel.
# loaded_model = mlflow.pyfunc.load_model(logged_model)

# # Predict on a Pandas DataFrame.
# import pandas as pd
# loaded_model.predict(test_raw_final)

array([110727.62550487, 168384.59474308, 181356.74058085, ...,
       156839.06690177, 109855.22622585, 228674.80599086])

***Linear Regression v3***

In [20]:
# from sklearn.preprocessing import StandardScaler

# rfe_features = ['OverallQual','1stFlrSF','2ndFlrSF','BsmtFullBath','GarageCars',
#     'BsmtQual_Gd','BsmtQual_TA','KitchenQual_Gd','KitchenQual_TA','Neighborhood']

# scaler = StandardScaler()

# X_train_scaled = pd.DataFrame(
#     scaler.fit_transform(X_train),
#     columns=X_train.columns
# )

# test_raw_scaled = pd.DataFrame(
#     scaler.transform(test_raw),  
#     columns=test_raw.columns
# )
# test_raw_final = test_raw_scaled[rfe_features]

# import mlflow
# logged_model = 'runs:/e6cfd6f8e7c34c3c87f95e53464ce627/final_model'

# # Load model as a PyFuncModel.
# loaded_model = mlflow.pyfunc.load_model(logged_model)

# # Predict on a Pandas DataFrame.
# import pandas as pd
# predictions = loaded_model.predict(test_raw_final)

**Decision Tree**

In [16]:
# # Scaling
# scaler = StandardScaler()
# X_train_scaled = pd.DataFrame(
#     scaler.fit_transform(X_train),
#     columns=X_train.columns
# )
# test_raw_scaled = pd.DataFrame(
#     scaler.transform(test_raw),
#     columns=test_raw.columns
# )

# import mlflow
# logged_model = 'runs:/7bd725d8375f4377bbc4c0daf39abe55/final_model_dt'

# # Load model as a PyFuncModel.
# loaded_model = mlflow.pyfunc.load_model(logged_model)

# # Predict on a Pandas DataFrame.
# import pandas as pd
# predictions = loaded_model.predict(test_raw_scaled)

**Random Forest**

In [26]:
# Scaling
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train),
    columns=X_train.columns
)
test_raw_scaled = pd.DataFrame(
    scaler.transform(test_raw),
    columns=test_raw.columns
)

import mlflow
logged_model = 'runs:/44e98da9148743b3a85f1fe3fb7d03a0/final_model_rf'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
predictions = loaded_model.predict(test_raw_scaled)

In [27]:
submission = pd.DataFrame({
    "Id": test_ids,
    "SalePrice": predictions
})

submission.to_csv("submission.csv", index=False)
